In [ ]:
import numpy as np
import cma
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor  
import torch


In [194]:
def true_objective_function(x):
    A = 10
    n = len(x)
    rastrigin = A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))
    return rastrigin

# 代理模型：高斯过程回归
class SurrogateModel:
    def __init__(self, kernel=None):
        self.model = TabPFNRegressor()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X,output_type="full")

    def update(self, X, y):
        self.fit(X, y)

# 代理模型辅助CMA-ES的目标函数评估
def surrogate_evaluation(x, surrogate_model, real_evaluation, X_train, y_train,count):

    # 首先使用代理模型进行评估
    predictions = surrogate_model.predict(x)
    criterion = predictions["criterion"]   # FullSupportBarDistribution 对象
    logits    = predictions["logits"] 
    pmf = logits.softmax(dim=-1).cpu().numpy()
    bin_edges = criterion.borders.detach().cpu().numpy()
    bin_midpoints = (bin_edges[1:]+bin_edges[0:-1])/2 # 每个bin的中点

    pred = np.sum(pmf*bin_midpoints,axis=1)# mean
    std = np.sqrt(np.sum(pmf*(bin_midpoints**2),axis=1)-np.sum(pmf*bin_midpoints,axis=1)**2) #std
    
    # 如果代理模型的预测不确定性较大，则使用真实目标函数进行评估
    for i in range(std.shape[0]):
        if std[i] > 20:  # 可调阈值，决定何时使用真实目标函数
            true_val = real_evaluation(x[i])
            pred[i] = true_val
            X_train = np.vstack([X_train, x[i]])  # 添加新解
            y_train = np.append(y_train, true_val)  # 添加真实目标值
            surrogate_model.update(X_train, y_train)  # 更新代理模型
            count += 1

    return pred,count
# 使用CMA-ES进行优化的主函数
def optimize_with_cma_es(dimension=6, popsize=10,max_iter=10, max_evals=500):
    # CMA-ES的初始配置
    es = cma.CMAEvolutionStrategy(np.random.randn(dimension), 0.33, {'popsize': popsize,'maxiter': max_iter, 'maxfevals': max_evals})

    # 训练数据初始化
    X_train = np.random.uniform(-5, 5, (10, dimension))  # 初始训练数据（随机生成）
    y_train = np.array([true_objective_function(x) for x in X_train])  # 真实目标函数值

    # 创建代理模型
    surrogate_model = SurrogateModel()

    # 用初始数据训练代理模型
    surrogate_model.fit(X_train, y_train)
    count = 0
    

    # 开始优化
    while not es.stop():
        solutions = es.ask()  # 获取新的候选解
        # 对每个候选解进行代理模型评估或实际评估

        fitness_values,count = surrogate_evaluation(np.vstack(solutions), surrogate_model, true_objective_function, X_train, y_train,count)
        es.tell(solutions, fitness_values)  # 更新CMA-ES的种群
        es.logger.add()  # 记录优化数据
        print(f"Best fitness so far: {es.result.fbest}, Best solution: {es.result.xbest}, true_f:{true_objective_function(es.result.xbest)}, count:{count}")
        es.disp()  # 显示当前状态
        

    return es.result.xbest, es.result.fbest

def cma_es(dimension=6, popsize=10,max_iter=10, max_evals=500):
    # CMA-ES的初始配置
    es = cma.CMAEvolutionStrategy(np.random.randn(dimension), 0.33, {'popsize': popsize,'maxiter': max_iter, 'maxfevals': max_evals})

    # 训练数据初始化
    X_train = np.random.uniform(-5, 5, (10, dimension))  # 初始训练数据（随机生成）
    y_train = np.array([true_objective_function(x) for x in X_train])  # 真实目标函数值
    # 开始优化
    while not es.stop():
        solutions = es.ask()  # 获取新的候选解
        # 对每个候选解进行代理模型评估或实际评估

        fitness_values = np.array([true_objective_function(x) for x in solutions])
        es.tell(solutions, fitness_values)  # 更新CMA-ES的种群
        es.logger.add()  # 记录优化数据
        print(f"Best fitness so far: {es.result.fbest}, Best solution: {es.result.xbest}, true_f:{true_objective_function(es.result.xbest)}")
        es.disp()  # 显示当前状态

    return es.result.xbest, es.result.fbest

In [200]:
# 运行优化
np.random.seed(123)
best_solution, best_fitness = optimize_with_cma_es()
print(f"Optimized solution: {best_solution}")
print(f"Best fitness value: {best_fitness}")

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 6 (seed=915653, Sun Apr  6 04:40:38 2025)
Best fitness so far: 32.97795867919922, Best solution: [-1.28529822  1.05632488 -0.09028371 -0.95982082 -0.2337683   1.89934801], true_f:32.97795938934626, count:10
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 3.297795867919922e+01 1.0e+00 3.14e-01  3e-01  3e-01 0:02.0
Best fitness so far: 32.97795867919922, Best solution: [-1.28529822  1.05632488 -0.09028371 -0.95982082 -0.2337683   1.89934801], true_f:32.97795938934626, count:19
    2     20 3.560767745971680e+01 1.2e+00 3.18e-01  3e-01  3e-01 0:03.8
Best fitness so far: 27.488208770751953, Best solution: [-0.80476546  0.10798715  0.16185149 -1.10040279 -0.91321881  1.8446241 ], true_f:27.4882083688084, count:29
    3     30 2.748820877075195e+01 1.3e+00 3.45e-01  3e-01  4e-01 0:05.6
Best fitness so far: 27.488208770751953, Best solution: [-0.80476546  0.10798715  0.16185149 -1.10040279 -0.91321881  1.844

In [ ]:
np.random.seed(123)
cma_es()

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 6 (seed=993181, Sun Apr  6 04:41:01 2025)
Best fitness so far: 39.39309128194073, Best solution: [-1.64233751  0.83751468 -0.0740262  -1.16058353 -0.14439338  1.97743692], true_f:39.39309128194073
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 3.939309128194073e+01 1.0e+00 3.24e-01  3e-01  3e-01 0:00.0
Best fitness so far: 39.39309128194073, Best solution: [-1.64233751  0.83751468 -0.0740262  -1.16058353 -0.14439338  1.97743692], true_f:39.39309128194073
    2     20 5.237087865455242e+01 1.3e+00 3.42e-01  3e-01  4e-01 0:00.0
Best fitness so far: 39.39309128194073, Best solution: [-1.64233751  0.83751468 -0.0740262  -1.16058353 -0.14439338  1.97743692], true_f:39.39309128194073
    3     30 5.083230692085447e+01 1.4e+00 4.00e-01  4e-01  5e-01 0:00.0
Best fitness so far: 39.39309128194073, Best solution: [-1.64233751  0.83751468 -0.0740262  -1.16058353 -0.14439338  1.97743692], true_f:39.3930912819407

(array([-1.88014018, -1.05499956,  0.11913776, -0.04905789, -0.084844  ,
         1.08418199]),
 15.041439776557887)

: 

In [129]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load Boston Housing data
df = fetch_openml(data_id=531, as_frame=True)  # Boston Housing dataset
X = df.data
y = df.target.astype(float)  # Ensure target is float for regression

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()  
regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test, output_type="full")
criterion = predictions["criterion"]   # FullSupportBarDistribution 对象
logits    = predictions["logits"] 
pmf = logits.softmax(dim=-1).cpu().numpy()
bin_edges = criterion.borders.detach().cpu().numpy()


In [60]:
predictions["mean"]

array([24.873093 , 31.856453 , 15.396772 , 23.41805  , 17.29749  ,
       21.805729 , 18.857859 , 14.769909 , 19.737427 , 21.02562  ,
       20.415693 , 18.096394 , 10.216825 , 22.258507 , 18.02958  ,
       24.740051 , 19.147125 , 10.0326805, 46.89821  , 15.431845 ,
       25.449898 , 27.60127  , 14.254143 , 22.447676 , 15.420011 ,
       16.274263 , 21.616476 , 13.701888 , 18.933498 , 20.373083 ,
       18.65894  , 24.108223 , 15.501996 , 21.447062 , 15.461344 ,
       16.290596 , 31.75388  , 19.416595 , 22.037195 , 24.474243 ,
       18.77325  , 30.173307 , 49.421864 , 18.714256 , 24.227812 ,
       14.95067  , 14.680441 , 25.678835 , 19.56859  , 23.571524 ,
       18.756157 , 34.605858 , 17.308681 , 27.459908 , 46.024044 ,
       22.106213 , 15.595837 , 31.501245 , 23.609722 , 17.120365 ,
       24.473541 , 34.763905 , 32.1237   , 16.735786 , 23.26847  ,
       15.453657 , 13.486303 , 23.725786 , 28.34314  , 14.302972 ,
       20.987864 , 24.382378 , 10.19917  , 19.527454 , 23.4058

In [ ]:
(bin_edges[np.argmax(pmf,axis=1)]+bin_edges[np.argmax(pmf,axis=1)+1])/2# mode

bin_midpoints = (bin_edges[1:]+bin_edges[0:-1])/2 # 每个bin的中点

np.sum(pmf*bin_midpoints,axis=1)# mean

np.sqrt(np.sum(pmf*(bin_midpoints**2),axis=1)-np.sum(pmf*bin_midpoints,axis=1)**2) #std

array([ 2.0770822,  2.4444137,  2.134947 ,  1.6379442,  2.19925  ,
        1.9356011,  1.8306106,  2.2353036,  1.7585763,  1.6466686,
        1.7854512,  1.6886389,  5.0218716,  1.8526006,  1.5921886,
        3.36091  ,  1.6940249,  3.141774 ,  3.6810722,  2.112505 ,
        1.9377835,  2.0648513,  1.5623388,  2.952851 ,  2.9720788,
        2.8066182,  1.7550151,  1.8933711,  1.7678274,  1.7149906,
        1.92035  ,  1.8616058,  4.9402375,  4.528771 ,  2.2117724,
        1.9040395,  3.3598838,  1.44855  ,  1.5546286,  1.6639158,
        2.4466288,  2.507849 ,  3.3758678,  1.6392479,  1.9957688,
        2.5947676,  1.5763255,  1.9162456,  1.7469281,  2.9560275,
        1.6286768,  2.7730303,  1.9685407,  2.4112039,  3.8150926,
        1.6341203,  2.1316423,  2.442353 ,  1.8650124,  1.7906141,
        2.1324549,  3.2129424,  3.4143753,  2.3476336,  2.536104 ,
        2.0219665,  2.0930467,  1.699358 ,  2.3012288,  2.3626323,
        1.4857416,  2.0156856,  2.1875088,  1.6709803,  1.8590